In [1]:
import glob, os, sys
import numpy as np
import csv
import cv2
import pickle
from sklearn.neighbors import NearestNeighbors
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
import tensorflow as tf
import PIL
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (Dropout, Dense, Softmax)
from tensorflow.keras.applications import mobilenet as _mobilenet
import numpy as np
from PIL import Image
import networkx as nx
from IPython.display import Image as iImage
from IPython.display import display
import math
import matplotlib.pyplot as plt
import ebooklib
from ebooklib import epub
import random
import re
import os
import nltk
from nltk.corpus import words
import gensim
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import words, stopwords, names
from sklearn.decomposition import PCA
import scipy.spatial
from matplotlib import cm
import matplotlib.image as mpimg
import pandas as pd
import jieba
from sklearn.metrics.pairwise import linear_kernel
from nltk import pos_tag
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import sklearn
from matplotlib.colors import LinearSegmentedColormap

In [2]:
%cd D:\rc11\collection\python

D:\rc11\collection\python


In [3]:
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:
def read_epub_book(epub_file, ID):
    book = epub.read_epub(epub_file)
    text = ""

    for item in book.get_items():
        if item.get_type() == ebooklib.ITEM_DOCUMENT:
            content = item.get_content().decode('utf-8')
            content = re.sub('<[^<]+?>', '', content)  # Remove HTML tags
            content = re.sub('\s+', ' ', content)  # Replace multiple whitespaces with a single space
            content = re.sub('\n', ' ', content)
            content = re.sub("&#13;", ' ', content)
            text += content
            #paragraphs.extend(content.strip().split("&#13;"))
    book_item = {'text': text, 'bookID':ID}
    return book_item

def read_multiple_epub(epub_files):
    IDs = [os.path.splitext(os.path.basename(e))[0] for e in epub_files]
    all_paragraphs = []
    for epub_file, ID in zip(epub_files, IDs):
        paragraphs = read_epub_paragraphs(epub_file, ID)
        all_paragraphs.extend(paragraphs)
    return all_paragraphs

def merge_strings_until_limit(strings, min_length, max_length, test_for_max = 0):
    merged_string = ""
    merged_strings = []

    for s in strings:
        if len(merged_string) <= min_length:# and len(s)<max_length:
            merged_string += s
        
        elif len(merged_string) > max_length and test_for_max<5:
            splitParagraph = merged_string.split('.')
            splitParagraphRePoint = []
            for sp in splitParagraph:
                splitParagraphRePoint.append(sp+'.')
            merged = merge_strings_until_limit(splitParagraphRePoint, min_length, max_length, test_for_max+1)
            merged_strings.extend(merged)
            merged_string = s
        else:
            merged_strings.append(merged_string)
            merged_string = s

    if merged_string:
        merged_strings.append(merged_string)

    return merged_strings

def read_epub_paragraphs(epub_file, ID):
    book = epub.read_epub(epub_file)
    paragraphs = []

    for item in book.get_items():
        if item.get_type() == ebooklib.ITEM_DOCUMENT:
            content = item.get_content().decode('utf-8')
            content = re.sub('<[^<]+?>', '', content)  # Remove HTML tags
            content = re.sub('\s+', ' ', content)  # Replace multiple whitespaces with a single space
            content = re.sub('\n', ' ', content)
            paragraphs.extend(content.strip().split("&#13;"))
    paragraphs = merge_strings_until_limit(paragraphs, 200, 1000)
    paragraphs = [{'paragraph':paragraphs[i], 'nr':i,'bookID':ID} for i in range(len(paragraphs)) if len(paragraphs[i])>50]
    return paragraphs[1:-1]

def preprocess(paragraphs):
    STOP_WORDS = set(stopwords.words("english"))
    LEMMATIZER = WordNetLemmatizer()
    STEMMER = PorterStemmer()
    
    processed_docs = [] 
    
    for paragraph in paragraphs:
        words = gensim.utils.simple_preprocess(paragraph, min_len=3, deacc=True)        
        lemmatized_words = [LEMMATIZER.lemmatize(word) for word in words]
        
        filtered_words = [word for word in lemmatized_words if (word not in STOP_WORDS) and (is_english_word(word))]
        
        stemmed_words = [STEMMER.stem(word) for word in filtered_words]
        
        processed_doc = " ".join(stemmed_words)
        processed_docs.append(processed_doc)  

    return processed_docs

def preprocess2(paragraphs):
    lemmatizer = WordNetLemmatizer()
    STOP_WORDS = set(stopwords.words("english"))

    processed_docs = []

    for paragraph in paragraphs:
        words = gensim.utils.simple_preprocess(paragraph, min_len=3, deacc=True)
        lemmatized_words = [lemmatizer.lemmatize(word) for word in words]
        filtered_words = [word for word in lemmatized_words if word not in STOP_WORDS]
        processed_docs.append(filtered_words)
    
    return processed_docs

In [5]:
english_words = set(words.words())

def is_english_word(word):
    return word in english_words

In [9]:
BOOK_DIR = r'D:\rc11\collection\text\landscape'
BOOK_FILES = []
BOOK_FILES.extend(glob.glob(os.path.join(BOOK_DIR, '*.epub')))

In [10]:
PARAGRAPHS = read_multiple_epub(BOOK_FILES)

D:\anaconda\lib\site-packages\ebooklib\epub.py:1395: UserWarning: In the future version we will turn default option ignore_ncx to True.
  warnings.warn('In the future version we will turn default option ignore_ncx to True.')


In [11]:
PARAGRAPHS

[{'paragraph': ' You may copy it, give it away or re-use it under the terms of the Project Gutenberg License included with this ebook or online at www.gutenberg.org. If you are not located in the United States, you will have to check the laws of the country where you are located before using this eBook.',
  'nr': 1,
  'bookID': 'pg1276-images-3'},
 {'paragraph': ' Title: The Rhythm of Life, and Other Essays Author: Alice Meynell Release date: April 1, 1998 [eBook #1276] Most recently updated: December 31, 2020 Language: English *** START OF THE PROJECT GUTENBERG EBOOK THE RHYTHM OF LIFE, AND OTHER ESSAYS *** Transcribed from the 1893 John Lane edition by David Price, email ccx074@coventry.',
  'nr': 2,
  'bookID': 'pg1276-images-3'},
 {'paragraph': 'ac.uk The Rhythm of Life and Other Essays Contents The Rhythm of Life Decivilised A Remembrance The Sun The Flower Unstable Equilibrium The Unit of the World By the Railway Side Pocket Vocabularies Pathos The Point of Honour Composure Dr.',

In [12]:
output_file_path = "PARAGRAPHS.pkl"
with open(output_file_path, "wb") as f:
    pickle.dump(PARAGRAPHS, f)

In [13]:
input_file_path = "PARAGRAPHS.pkl"
with open(input_file_path, "rb") as f:
    PARAGRAPHS = pickle.load(f)

In [14]:
len(PARAGRAPHS)

37232

In [15]:
PREPROCESSED = preprocess([p['paragraph'] for p in PARAGRAPHS])

In [16]:
PREPROCESSED2 = preprocess2([p['paragraph'] for p in PARAGRAPHS])

In [21]:
with open('PREPROCESSED.txt', 'w') as f:
    for line in PREPROCESSED:
        f.write(line + '\n')

In [22]:
PREPROCESSED = []

with open('PREPROCESSED.txt', 'r') as f:
    for line in f:
        PREPROCESSED.append(line.strip())

In [23]:
with open('PREPROCESSED2.txt', 'w') as f:
    for line in PREPROCESSED:
        f.write(line + '\n')

In [24]:
PREPROCESSED2 = []
with open('PREPROCESSED2.txt', 'r', encoding='utf-8') as f: 
    for line in f:  
        PREPROCESSED2.append(line.strip())

In [25]:
len(PREPROCESSED)

37232

tfidf

In [26]:
VECTORIZER_TFIDF = TfidfVectorizer(min_df=2)
TFIDF_MATRIX = VECTORIZER_TFIDF.fit_transform(PREPROCESSED)

In [27]:
def search_tfidf(query, k=1):
    model = NearestNeighbors(n_neighbors = k,
                            metric = 'cosine',
                            algorithm = 'brute',
                            n_jobs = -1)
    model.fit(TFIDF_MATRIX)
    qv = get_tfidf_vector(query)
    neighbours = model.kneighbors(qv, k, return_distance = False)[0]
    paragraphs = [PARAGRAPHS[i] for i in neighbours]
    return paragraphs

def get_tfidf_vector(query):
    query_vector = VECTORIZER_TFIDF.transform([query])
    return query_vector

In [28]:
import pickle
with open('database01_book_TFIDF.pkl', 'wb') as f:
    pickle.dump(TFIDF_MATRIX, f)

In [29]:
import pickle
with open('database01_book_TFIDF.pkl', 'rb') as f:
    TFIDF_MATRIX = pickle.load(f)

In [30]:
search_tfidf('street',9)

[{'paragraph': ' That is a Parisian street. Go on now to the next street, and you find precisely the same conditions repeated—the streets that cross are similar, those that radiate the same. Some are short, others long, some wide, some narrow; they are all geometry and white paint.',
  'nr': 1411,
  'bookID': 'pg6164-images-3'},
 {'paragraph': ' H. Hunnewell estate at Wellesley, and for many other private clients. 1882-1883: Winter In Brookline, at Taylor house on Dudley Street. 1883: Bought house at 99 Warren Street, Brookline, which became permanent home and office.',
  'nr': 150,
  'bookID': 'pg70767-images-3'},
 {'paragraph': ' On the contrary, the present lay-out of either town shows four streets meeting at an open space or Carfax in the centre of the city: no trace remains above ground of the closed forum, which at Silchester and Corstopitum formed the centre of the plan and directed the course of the streets.',
  'nr': 232,
  'bookID': 'pg60518-images-3'},
 {'paragraph': ' They 

Doc2Vec

In [31]:
from gensim.models.doc2vec import TaggedDocument, Doc2Vec

In [32]:
DOC2VEC_DOCUMENTS = [TaggedDocument(doc, [i]) for i, doc in enumerate(PREPROCESSED2)]

DOC2VEC_MODEL = Doc2Vec(vector_size=250, min_count=0, alpha=0.025, min_alpha=0.025, epochs=100)
DOC2VEC_MODEL.build_vocab(DOC2VEC_DOCUMENTS)
DOC2VEC_MODEL.train(DOC2VEC_DOCUMENTS, total_examples=DOC2VEC_MODEL.corpus_count, epochs=100)

In [33]:
DOC2VEC_MODEL.save("database01_book_Doc2Vec.bin")

In [34]:
loaded_model1 = Doc2Vec.load("database01_book_Doc2Vec.bin")
DOC2VEC_MODEL = loaded_model1

In [35]:
def get_doc2vec_vector(query):
    tokens = preprocess2([query])[0]
    vector = DOC2VEC_MODEL.infer_vector(tokens)
    return vector

In [36]:
def search_doc2vec(query, k=1):
    vectorQuery = get_doc2vec_vector(query)
    neighbors = DOC2VEC_MODEL.dv.most_similar(positive=[vectorQuery], topn=k)
    paragraphs = [PARAGRAPHS[n[0]] for n in neighbors]
    return paragraphs

In [44]:
query = 'urban landscaping environment'

In [45]:
search_doc2vec(query,9)

[{'paragraph': ' Caverns, too, and chasms are there, dark, deep, chilly, and damp, where the toad, the lizard and snake, and strange families of insects, are perpetually multiplying and actually seeming to enjoy their loathsome lives; the Black Chasm, the Gray Chasm, and the Devil’s Chamber, with perpendicular walls of 50 twice the height of a tall mast, and with a wainscoting of pines and hemlocks, that have “braved a thousand years the battle and the breeze.',
  'nr': 198,
  'bookID': 'pg53942-images-3'},
 {'paragraph': " So you may define life by its attached negative, death; and still more by its attached positive, birth. But I won't be plagued any more about this, just now; if you choose to think the crystals alive, do, and welcome.",
  'nr': 2929,
  'bookID': 'pg26716-images-3'},
 {'paragraph': ' We bent to the west, turned the flank of a high range, and after half an hour’s steady trotting, reached a side-valley or cul-de-sac, branching off from the main wady. A herd of loose ca

In [46]:
sentence_vectors1 = np.array([DOC2VEC_MODEL.dv[i] for i in range(len(DOC2VEC_MODEL.dv))])

In [47]:
sentence_vectors1.shape

(37232, 250)

In [48]:
featureTextPairs1 = []  

for i in range(len(sentence_vectors1)):
    featureText = {}
    featureText['sentence'] = PARAGRAPHS[i]
    featureText['feature'] = sentence_vectors1[i]
    featureTextPairs1.append(featureText)

print(len(featureTextPairs1))

37232


In [49]:
import pickle
output_file_path = "featureTextPairs1.pkl"
with open(output_file_path, "wb") as f:
    pickle.dump(featureTextPairs1, f)
print(f"Feature-image pairs saved to {output_file_path}")

Feature-image pairs saved to featureTextPairs1.pkl


In [50]:
import pickle
input_file_path = "featureTextPairs1.pkl"
with open(input_file_path, "rb") as f:
    featureTextPairs1 = pickle.load(f)